In [0]:
dbutils.widgets.text("bronze_schema","")
dbutils.widgets.text("bronze_tbl","")
dbutils.widgets.text("last_insert_dttm","")
dbutils.widgets.text("load_id","")
dbutils.widgets.text("storage_account_name","")
dbutils.widgets.text("source_table_name","")

In [0]:
bronze_schema = dbutils.widgets.get("bronze_schema")
bronze_tbl = dbutils.widgets.get("bronze_tbl")
last_insert_dttm = dbutils.widgets.get("last_insert_dttm")
load_id = dbutils.widgets.get("load_id")
storage_account_name = dbutils.widgets.get("storage_account_name")
source_table_name = dbutils.widgets.get("source_table_name")

In [0]:
#reading source file into df

source_file_path = f'abfss://landing@{storage_account_name}.dfs.core.windows.net/mysql_files/{source_table_name}'

df = spark.read.format("parquet").load(source_file_path)

In [0]:
#checking if the column names and first row values are same in the table

from pyspark.sql.functions import *


first_column_name = df.columns[0]
first_column_first_row_value = df.select(df.columns[0]).first()[0]

#removing the row if it's same as the column name

if first_column_name == first_column_first_row_value:
    df = df.filter(col(first_column_name) != first_column_first_row_value)
    source_file_records_count = df.count()
else:
    source_file_records_count = df.count()

#create temporary view for the source file
df.createOrReplaceTempView(f"{source_table_name}_temp")


In [0]:
spark.sql(f"""
          INSERT INTO {bronze_schema}.{bronze_tbl}
          SELECT *, '{last_insert_dttm}' AS last_insert_dttm, '{load_id}' AS load_id FROM {source_table_name}_temp
          """)

In [0]:
bronze_tbl_records_count = spark.sql(f"SELECT COUNT(*) FROM {bronze_schema}.{bronze_tbl} WHERE load_id = '{load_id}'").collect()[0][0]

In [0]:
#returning records count to ADF
dbutils.notebook.exit([source_file_records_count,bronze_tbl_records_count])